In [1]:
from tensorflow.keras.utils import set_random_seed, plot_model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras.datasets.cifar10 import load_data
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16
import tensorflow as tf

In [2]:
# W,b 고정하도록 seed 설정
set_random_seed(11)

In [3]:
(x_train, y_train), (x_test, y_test) = load_data()

In [4]:
print(f' x_train.shape: {x_train.shape}, y_train.shape : {y_train.shape}')
print(f' x_test.shape: {x_test.shape}, y_test.shape : {y_test.shape}')

 x_train.shape: (50000, 32, 32, 3), y_train.shape : (50000, 1)
 x_test.shape: (10000, 32, 32, 3), y_test.shape : (10000, 1)


In [5]:
# 스켈일링 => 픽셀 /255.0
x_train=x_train/255.0
x_test=x_test/255.0

In [6]:
# Conv 레이어는 이미지의 채널 정보까지 입력
x_train=x_train.reshape(-1, 32, 32, 3)
x_test=x_test.reshape(-1, 32, 32, 3)

In [7]:
print(f'x_train.shape : {x_train.shape}, x_test.shape : {x_test.shape}')

x_train.shape : (50000, 32, 32, 3), x_test.shape : (10000, 32, 32, 3)


In [8]:
# 검증용 데이터 준비
from sklearn.model_selection import train_test_split

In [9]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                  stratify=y_train,
                                                  random_state=11)

In [10]:
print(f'x_train.shape : {x_train.shape}\nx_val.shape : {x_val.shape}\nx_test.shape : {x_test.shape}')

x_train.shape : (37500, 32, 32, 3)
x_val.shape : (12500, 32, 32, 3)
x_test.shape : (10000, 32, 32, 3)


In [11]:
model_vgg = VGG16(weights='imagenet', include_top=False)

In [12]:
for layer in model_vgg.layers:
    layer.trainable = False

In [13]:
model_vgg.layers[-8:]

In [14]:
for layer in model_vgg.layers[-8:]:
    layer.trainable = True

In [15]:
inputs = model_vgg.output
x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(10, activation='softmax')(x)

In [16]:
new_model = tf.keras.models.Model(model_vgg.input, x)
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [17]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [18]:
rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, mode='min', verbose=1)
ely_cb = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)

In [19]:
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
x_train, y_train, x_val, y_val

(array([[[[0.68627451, 0.76470588, 0.81568627],
          [0.89411765, 0.9254902 , 0.94117647],
          [0.95294118, 0.96470588, 0.96078431],
          ...,
          [0.94509804, 0.97647059, 0.98431373],
          [0.94509804, 0.97647059, 0.98431373],
          [0.96470588, 0.99607843, 1.        ]],
 
         [[0.61960784, 0.69411765, 0.78039216],
          [0.79215686, 0.83137255, 0.8745098 ],
          [0.92941176, 0.95686275, 0.97254902],
          ...,
          [0.95294118, 0.98431373, 0.98431373],
          [0.94901961, 0.98431373, 0.98039216],
          [0.96470588, 1.        , 0.99607843]],
 
         [[0.60784314, 0.6745098 , 0.78823529],
          [0.65098039, 0.70196078, 0.77647059],
          [0.80784314, 0.85098039, 0.89411765],
          ...,
          [0.9372549 , 0.97254902, 0.96470588],
          [0.93333333, 0.96862745, 0.96078431],
          [0.94901961, 0.98431373, 0.97647059]],
 
         ...,
 
         [[0.39607843, 0.43529412, 0.49803922],
          [0.36862

In [21]:
# steps_per_epoch = 391
new_model.fit(x_train, y_train,
                        epochs = 5,
                        validation_data = (x_val, y_val))

Epoch 1/5
1172/1172 [==============================] - 855s 728ms/step - loss: 1.7372 - accuracy: 0.3003 - val_loss: 1.3271 - val_accuracy: 0.4833
Epoch 2/5
1172/1172 [==============================] - 856s 730ms/step - loss: 1.2690 - accuracy: 0.5337 - val_loss: 1.0365 - val_accuracy: 0.6409
Epoch 3/5
1172/1172 [==============================] - 881s 752ms/step - loss: 1.0224 - accuracy: 0.6526 - val_loss: 0.9469 - val_accuracy: 0.6785
Epoch 4/5
1172/1172 [==============================] - 876s 747ms/step - loss: 0.9168 - accuracy: 0.6924 - val_loss: 0.9142 - val_accuracy: 0.6949
Epoch 5/5
1172/1172 [==============================] - 894s 763ms/step - loss: 0.8281 - accuracy: 0.7278 - val_loss: 0.8537 - val_accuracy: 0.7198
